In [1]:
# main
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchmetrics import R2Score


# implementation
from tools import make_regression_data, RegressionDataset, training_model
from optimizers import FiniteTimeOptimizer

# graphics
import matplotlib.pyplot as plt


torch.random.manual_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"

## Data

In [2]:
NUMBER_OF_FEATURES = 10
X_train, X_test, y_train, y_test = make_regression_data(number_samples=100,
                                                        number_features=NUMBER_OF_FEATURES,
                                                        noise_value=5.5)
print(f'shape of train: {X_train.shape, y_train.shape}\nshape of test: {X_test.shape, y_test.shape}')

shape of train: (torch.Size([80, 10]), torch.Size([80, 1]))
shape of test: (torch.Size([20, 10]), torch.Size([20, 1]))


In [3]:
train_dataset = RegressionDataset(features=X_train,
                                  labels=y_train)
test_dataset = RegressionDataset(features=X_test,
                                 labels=y_test)
print(f'example of train sample:\n {train_dataset[19]}')

example of train sample:
 (tensor([-0.5173, -0.5229,  1.5796,  2.2989,  1.4534, -0.3628, -0.2818,  1.4093,
        -0.4455, -0.4202]), tensor([185.5551]))


In [4]:
BATCH_SIZE = 10

train_dataloader = DataLoader(dataset=train_dataset,
                              shuffle=True,
                              batch_size=BATCH_SIZE)
test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=BATCH_SIZE)
batch_example_features, batch_example_labels  = next(iter(train_dataloader))
print('shape of batch: features - {} and labels - {}'.format(batch_example_features.shape, batch_example_labels.shape))

shape of batch: features - torch.Size([10, 10]) and labels - torch.Size([10, 1])


## Model

In [5]:
simple_model = nn.Linear(in_features=10, out_features=1, bias=False)

## Loss & metric & optimizer

In [6]:
loss_fn = nn.MSELoss()

metric_fn = R2Score()

N_OF_BATCHES = 15
optimizer = FiniteTimeOptimizer(params=simple_model.parameters(),
                                lr=0.01,
                                n_of_batches=N_OF_BATCHES)

## Train

In [7]:
batch_number = 0
while batch_number < N_OF_BATCHES:
    simple_model.train()
    for X_batch, y_batch in train_dataloader:
        predicted = simple_model(X_batch)
        batch_number += 1
        determinant = torch.det(X_batch)
        inverse_batch = torch.linalg.inv(X_batch)
        adjoin = determinant * inverse_batch
        loss = loss_fn(adjoin @ predicted, adjoin @ y_batch)

        # zero gradients
        optimizer.zero_grad()

        # backpropagation (compute gradient)
        loss.backward()

        # parameters update
        optimizer.step(det_batch=determinant, t=batch_number)

        if batch_number == N_OF_BATCHES:
            print('finite-time estimation has been applied')
            break
    # validation
    with torch.inference_mode():
        metric, loss = 0.0, 0.0
        for X_batch, y_batch in test_dataloader:
            predicted = simple_model(X_batch)
            loss += loss_fn(predicted, y_batch)
            metric += metric_fn(predicted, y_batch)
        metric /= len(test_dataloader)
        loss /= len(test_dataloader)

        print(f"After {batch_number} batches")
        print(f"metric = {metric:.3f}")
        print(f"loss = {loss:.3f}")

After 8 batches
metric = 0.964
loss = 2330.733
finite-time estimation has been applied
After 15 batches
metric = 0.991
loss = 613.749


/home/ffrankusha/work/university/research-work/optimizers/Finite_Time_Optimizer.py:91: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral, error = quad(func=square_det,


## Test

In [8]:
with torch.inference_mode():
    metric, loss = 0.0, 0.0
    for X_batch, y_batch in test_dataloader:
        predicted = simple_model(X_batch)
        loss += loss_fn(predicted, y_batch)
        metric += metric_fn(predicted, y_batch)
    metric /= len(test_dataloader)
    loss /= len(test_dataloader)

    print(f"R2 = {metric:.3f}")
    print(f"loss = {loss:.3f}")

R2 = 0.991
loss = 613.749
